<a href="https://colab.research.google.com/github/Ann-Bond-coder/Artificial-intelligence/blob/main/Bondarenko_AI(lab7).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Виконати імпорт необхідних бібліотек.
import pandas as pd
import numpy as np
import os

from google.colab import drive
from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [13]:
drive.mount('/content/gdrive')
data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/superstore.csv', encoding='latin-1', parse_dates=['Order Date'], decimal=',', delimiter=';')
data.head()

Mounted at /content/gdrive


<ipython-input-13-90397d25a415>:2: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  data = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/data/superstore.csv', encoding='latin-1', parse_dates=['Order Date'], decimal=',', delimiter=';')


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11,11.11.2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11,11.11.2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06,16.06.2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10,18.10.2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10,18.10.2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [14]:
# 3. Вивести детальну інформацію про всі властивості датасету
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Row ID         9994 non-null   int64         
 1   Order ID       9994 non-null   object        
 2   Order Date     9994 non-null   datetime64[ns]
 3   Ship Date      9994 non-null   object        
 4   Ship Mode      9994 non-null   object        
 5   Customer ID    9994 non-null   object        
 6   Customer Name  9994 non-null   object        
 7   Segment        9994 non-null   object        
 8   Country        9994 non-null   object        
 9   City           9994 non-null   object        
 10  State          9994 non-null   object        
 11  Postal Code    9994 non-null   int64         
 12  Region         9994 non-null   object        
 13  Product ID     9994 non-null   object        
 14  Category       9994 non-null   object        
 15  Sub-Category   9994 n

In [15]:
# 4. Створити новий датафрейм з чисельними даними
numeric_data = data.select_dtypes(include=['float64', 'int64'])

numeric_data

,Row ID,Postal Code,Sales,Quantity,Discount,Profit
0,1,42420,261.9600,2,0.00,41.9136
1,2,42420,731.9400,3,0.00,219.5820
2,3,90036,14.6200,2,0.00,6.8714
3,4,33311,957.5775,5,0.45,-383.0310
4,5,33311,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...
9989,9990,33180,25.2480,3,0.20,4.1028
9990,9991,92627,91.9600,2,0.00,15.6332
9991,9992,92627,258.5760,2,0.20,19.3932
9992,9993,92627,29.6000,4,0.00,13.3200


In [25]:
#6. Отримання назв штатів за порядковими номерами 17, 18, 19
unique_states = data['State'].unique()
state_names = unique_states[[17, 18, 19]]
print(state_names)

['Virginia' 'Tennessee' 'Alabama']


In [18]:
# 5. Фільтрація даних та виведення унікальних значень
# Для стовбця Ship Mode - обрати Standard Class; для стовпця Segment обрати Corporate. Вивести унікальні значення стовбця State.
filtered_data = data[(data['Ship Mode'] == 'Standard Class') & (data['Segment'] == 'Corporate') & (data['State'].isin(state_names))] 
filtered_data

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
118,119,US-2015-136476,2015-05-04,10.04.2015,Standard Class,GG-14650,Greg Guthrie,Corporate,United States,Bristol,...,37620,South,OFF-BI-10003650,Office Supplies,Binders,GBC DocuBind 300 Electric Binding Machine,157.794,1,0.7,-115.7156
204,205,CA-2017-126382,2017-03-06,07.06.2017,Standard Class,HK-14890,Heather Kirkland,Corporate,United States,Franklin,...,37064,South,FUR-FU-10002960,Furniture,Furnishings,"Eldon 200 Class Desk Accessories, Burgundy",35.168,7,0.2,9.6712
865,866,CA-2014-149020,2014-10-01,15.01.2014,Standard Class,AJ-10780,Anthony Jacobs,Corporate,United States,Springfield,...,22153,South,OFF-LA-10004272,Office Supplies,Labels,Avery 482,2.890,1,0.0,1.3583
866,867,CA-2014-149020,2014-10-01,15.01.2014,Standard Class,AJ-10780,Anthony Jacobs,Corporate,United States,Springfield,...,22153,South,FUR-FU-10000965,Furniture,Furnishings,"Howard Miller 11-1/2"" Diameter Ridgewood Wall ...",51.940,1,0.0,21.2954
994,995,CA-2014-117639,2014-05-21,25.05.2014,Standard Class,MW-18235,Mitch Willingham,Corporate,United States,Virginia Beach,...,23464,South,OFF-BI-10003925,Office Supplies,Binders,Fellowes PB300 Plastic Comb Binding Machine,2715.930,7,0.0,1276.4871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9265,9266,US-2017-128118,2017-11-24,01.12.2017,Standard Class,MY-17380,Maribeth Yedwab,Corporate,United States,Memphis,...,38109,South,OFF-EN-10003448,Office Supplies,Envelopes,"Peel & Seel Recycled Catalog Envelopes, Brown",64.848,7,0.2,24.3180
9489,9490,US-2016-102232,2016-05-08,11.08.2016,Standard Class,MF-17665,Maureen Fritzler,Corporate,United States,Huntsville,...,35810,South,OFF-AR-10000538,Office Supplies,Art,"Boston Model 1800 Electric Pencil Sharpener, Gray",197.050,7,0.0,59.1150
9732,9733,CA-2014-114321,2014-08-20,25.08.2014,Standard Class,NC-18535,Nick Crebassa,Corporate,United States,Hampton,...,23666,South,FUR-CH-10001797,Furniture,Chairs,"Safco Chair Connectors, 6/Carton",500.240,13,0.0,145.0696
9733,9734,CA-2014-114321,2014-08-20,25.08.2014,Standard Class,NC-18535,Nick Crebassa,Corporate,United States,Hampton,...,23666,South,OFF-PA-10000246,Office Supplies,Paper,Riverleaf Stik-Withit Designer Note Cubes,20.120,2,0.0,9.2552


In [24]:
filtered_data['State'].tail()

9265    Tennessee
9489      Alabama
9732     Virginia
9733     Virginia
9734     Virginia
Name: State, dtype: object

In [19]:
# 7. Розбиття на навчальну та тестову множину
train_data, test_data = train_test_split(data, test_size=0.2, random_state=17)

In [20]:
# 8. Створення екземпляра класу MinMaxScaler
scaler = MinMaxScaler()

In [21]:
# 9. Навчання скалера на навчальній множині
scaler.fit(train_data[['Sales', 'Discount', 'Profit']])

MinMaxScaler()

In [22]:
# 10. Нормалізація даних навчальної та тестової множини
train_normalized = scaler.transform(train_data[['Sales', 'Discount', 'Profit']])
test_normalized = scaler.transform(test_data[['Sales', 'Discount', 'Profit']])

In [23]:
print(train_normalized)
print(test_normalized)

[[0.00209965 0.25       0.44031972]
 [0.00067656 0.25       0.44023628]
 [0.03469541 0.         0.45414577]
 ...
 [0.00054439 0.25       0.44006372]
 [0.05298746 0.25       0.46899939]
 [0.00676331 0.25       0.44345481]]
[[7.26856340e-03 0.00000000e+00 4.43299693e-01]
 [4.55869935e-05 1.00000000e+00 4.39852282e-01]
 [3.39034711e-02 2.50000000e-01 4.59198742e-01]
 ...
 [4.09222779e-04 7.50000000e-01 4.39611561e-01]
 [1.74379085e-03 2.50000000e-01 4.40864819e-01]
 [6.88487288e-04 1.00000000e+00 4.38290011e-01]]
